In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
import streamlit as st

In [44]:
df = pd.read_csv('1429_1.csv')
df.head()


C:\Users\moham\AppData\Local\Temp\ipykernel_15120\3933389454.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('1429_1.csv')


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [9]:
print(df.shape)
print(df.columns)
print(df.info())
print(df.isnull().sum())


(34660, 21)
Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    34660 non-null  object 
 1   name                  27900 non-null  object 
 2   asins                 34658 non-null  object 
 3   brand                 34660 non-null  object 
 4   categories            34660 non-null  object 
 5   keys                  34660 non-null  object 
 6   manufacturer          34660 non-null  object 
 7   revi

In [10]:
# Keep only useful columns
df = df[['name', 'brand', 'reviews.rating', 'reviews.text', 'reviews.doRecommend']]

# Drop rows with null in important columns
df.dropna(subset=['reviews.rating', 'reviews.text'], inplace=True)

# Fill missing recommendations with False
df['reviews.doRecommend'].fillna(False, inplace=True)

# Convert rating to int
df['reviews.rating'] = df['reviews.rating'].astype(int)


C:\Users\moham\AppData\Local\Temp\ipykernel_15120\2591030950.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews.doRecommend'].fillna(False, inplace=True)
C:\Users\moham\AppData\Local\Temp\ipykernel_15120\2591030950.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['reviews.doRecommend'].fillna

In [11]:
# Create binary label for good/bad product
df['is_good_product'] = df['reviews.rating'].apply(lambda x: 1 if x >= 4 else 0)

# Label best products (rating = 5 and recommended)
df['is_best_product'] = ((df['reviews.rating'] == 5) & (df['reviews.doRecommend'] == True)).astype(int)


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_features=3000)
X_text = tfidf.fit_transform(df['reviews.text'])


In [13]:
from scipy.sparse import hstack

X = hstack([X_text])
y = df['is_good_product']  # Change to other labels if needed


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9374819520646838
              precision    recall  f1-score   support

           0       0.52      0.08      0.14       435
           1       0.94      1.00      0.97      6491

    accuracy                           0.94      6926
   macro avg       0.73      0.54      0.55      6926
weighted avg       0.91      0.94      0.92      6926



In [16]:
best_products = df[df['is_best_product'] == 1]
print(best_products[['name', 'brand']].drop_duplicates().head(10))


                                                    name   brand
0      All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...  Amazon
2814   Kindle Oasis E-reader with Leather Charging Co...  Amazon
2905   All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...  Amazon
3052   Fire HD 8 Tablet with Alexa, 8 HD Display, 32 ...  Amazon
3065   Amazon 5W USB Official OEM Charger and Power A...  Amazon
3479   Amazon Kindle Fire Hd (3rd Generation) 8gb,,,\...  Amazon
3480   Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...  Amazon
14448  Kindle Oasis E-reader with Leather Charging Co...  Amazon
14452  Amazon - Kindle Voyage - 4GB - Wi-Fi + 3G - Bl...  Amazon
14478  Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...  Amazon


In [17]:
def recommend_products(brand):
    return best_products[best_products['brand'].str.contains(brand, na=False)][['name', 'reviews.rating']].head(5)

recommend_products('Amazon')


,name,reviews.rating
0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5
1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5
2,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5
4,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5
5,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(best_products['reviews.text'].fillna(""))

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations(index, cosine_sim=cosine_sim):
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    product_indices = [i[0] for i in sim_scores]
    return best_products[['name', 'brand']].iloc[product_indices]


In [26]:
from textblob import TextBlob

# Function to get sentiment polarity
def get_sentiment(text):
    if pd.isna(text):
        return 0
    return TextBlob(text).sentiment.polarity

# Apply on the reviews text
df['sentiment_score'] = df['reviews.text'].apply(get_sentiment)

# Define if user likely to buy (True/False)
df['likely_to_buy'] = ((df['sentiment_score'] > 0.3) & (df['reviews.rating'] >= 4))


In [29]:
print(df.columns)


Index(['name', 'brand', 'reviews.rating', 'reviews.text',
       'reviews.doRecommend', 'is_good_product', 'is_best_product',
       'sentiment_score', 'likely_to_buy'],
      dtype='object')


In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Fill missing values safely
best_products.loc[:, 'reviews.text'] = best_products['reviews.text'].fillna('')
best_products.loc[:, 'reviews.rating'] = best_products['reviews.rating'].fillna(0)

# TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')

def recommend_products(brand_name, top_n=5, rating_weight=0.6, text_weight=0.4):
    # Filter products for the given brand
    brand_df = best_products[best_products['brand'].str.contains(brand_name, na=False, case=False)].reset_index(drop=True)
    
    if brand_df.empty:
        return f"No products found for brand: {brand_name}"
    
    # TF-IDF on this brand only
    tfidf_matrix = tfidf.fit_transform(brand_df['reviews.text'])

    # Calculate average TF-IDF vector and convert to ndarray
    brand_mean_vector = np.asarray(tfidf_matrix.mean(axis=0)).reshape(1, -1)

    # Cosine similarity of each product's review vector with the average
    sim_scores = cosine_similarity(tfidf_matrix, brand_mean_vector).flatten()

    # Normalize ratings
    norm_ratings = brand_df['reviews.rating'] / brand_df['reviews.rating'].max()

    # Final score = weighted rating + weighted similarity
    final_scores = (rating_weight * norm_ratings) + (text_weight * sim_scores)
    brand_df['final_score'] = final_scores

    # Return top N recommended products
    return brand_df.sort_values(by='final_score', ascending=False)[['name', 'brand', 'reviews.rating', 'final_score']].head(top_n)

# Example usage:
recommend_products("Amazon")


,name,brand,reviews.rating,final_score
10160,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",Amazon,5,0.810952
13733,"Echo (White),,,\r\nEcho (White),,,",Amazon,5,0.805315
6545,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",Amazon,5,0.801526
1463,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,5,0.800652
753,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,5,0.800547


In [38]:
import pandas as pd
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Step 0: Assume best_products is already defined and filtered

# Step 1: Ensure 'reviews.text' has no nulls
best_products.loc[:, 'reviews.text'] = best_products['reviews.text'].fillna("")

# Step 2: Calculate sentiment score
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

best_products.loc[:, 'sentiment_score'] = best_products['reviews.text'].apply(get_sentiment)

# Step 3: Create binary label
best_products.loc[:, 'likely_to_buy'] = ((best_products['reviews.rating'] >= 4) &
                                         (best_products['sentiment_score'] > 0)).astype(int)

# Step 4: Prepare data for model
X = best_products[['reviews.rating', 'sentiment_score']]
y = best_products['likely_to_buy']

# Step 5: Split into training/testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Train the classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Step 7: Predict & Evaluate
y_pred = clf.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


✅ Accuracy: 0.9924989284183455
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       360
           1       1.00      0.99      1.00      4306

    accuracy                           0.99      4666
   macro avg       0.96      1.00      0.97      4666
weighted avg       0.99      0.99      0.99      4666



In [41]:
# 👤 Dummy product input (example)
dummy_rating = 5
dummy_review = "Absolutely love it! Works like a charm and arrived fast."

# 💬 Sentiment score for the dummy review
dummy_sentiment = get_sentiment(dummy_review)

# 📦 Format as input for model
dummy_input = pd.DataFrame([[dummy_rating, dummy_sentiment]], columns=['reviews.rating', 'sentiment_score'])

# 🔍 Predict
prediction = clf.predict(dummy_input)[0]
print("✅ Prediction:", "Likely to Buy" if prediction == 1 else "Not Likely to Buy")


✅ Prediction: Likely to Buy


In [42]:
import joblib

# Save the model
joblib.dump(clf, "buy_predictor_model.pkl")

# Optional: Save the TF-IDF vectorizer if needed for text input later
# joblib.dump(tfidf, "tfidf_vectorizer.pkl")


['buy_predictor_model.pkl']

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Sample dataset
data = pd.DataFrame({
    'brand': ['Amazon', 'Sony', 'Samsung', 'Apple', 'OnePlus', 'Xiaomi', 'Sony', 'Amazon'],
    'category': ['Electronics', 'Electronics', 'Electronics', 'Electronics', 'Electronics', 'Electronics', 'Electronics', 'Electronics'],
    'rating': [4.3, 3.2, 4.7, 2.1, 4.8, 3.5, 4.6, 2.0],
    'review_text': [
        'Great sound quality and build. Worth the price.',
        'Average sound quality, not worth the price.',
        'Excellent phone with great battery life.',
        'Overpriced and poor camera.',
        'Fantastic performance and smooth UI.',
        'Decent phone for the price.',
        'Superb noise cancellation and comfort.',
        'Disappointing build quality.'
    ],
    'is_best': [1, 0, 1, 0, 1, 0, 1, 0],
    'will_buy': [1, 0, 1, 0, 1, 1, 1, 0]
})

# Encode categorical data
label_encoders = {}
for column in ['brand', 'category', 'review_text']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Features and labels
X = data[['brand', 'category', 'rating', 'review_text']]
y = data['will_buy']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification = classification_report(y_test, y_pred)

accuracy, classification


(1.0,
 '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00         1\n           1       1.00      1.00      1.00         1\n\n    accuracy                           1.00         2\n   macro avg       1.00      1.00      1.00         2\nweighted avg       1.00      1.00      1.00         2\n')